In [ ]:
# inference.ipynb

# Install required packages
!pip install -q opencv-python Pillow pandas tensorflow

# Imports
import os
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from tensorflow.keras.models import load_model

# Paths and constants
BASE_PATH = '/kaggle/input/soil-classification-part-2/soil_competition-2025'
TEST_DIR = os.path.join(BASE_PATH, 'test')
TEST_CSV = os.path.join(BASE_PATH, 'test_ids.csv')
OUTPUT_CSV = '/kaggle/working/submission.csv'
MODEL_PATH = '/kaggle/working/soil_classifier_model.h5'  # Path to saved model
IMAGE_SIZE = (128, 128)
SUPPORTED_EXTS = ['.jpg', '.jpeg', '.png', '.webp', '.gif', '.ipeg']

# Function to load image and resize
def load_image(path, size):
    ext = os.path.splitext(path)[1].lower()
    try:
        if ext == ".gif":
            img = Image.open(path).convert('RGB').resize(size)
            return np.array(img)
        img = cv2.imread(path)
        if img is None:
            return None
        return cv2.resize(img, size)
    except:
        return None

# Load the trained model
model = load_model(MODEL_PATH)

# Load test dataframe
test_df = pd.read_csv(TEST_CSV)

results = []

for row in test_df.itertuples():
    img_path = os.path.join(TEST_DIR, row.image_id)
    if os.path.exists(img_path) and os.path.splitext(img_path)[1].lower() in SUPPORTED_EXTS:
        img = load_image(img_path, IMAGE_SIZE)
        if img is not None:
            img = img.astype("float32") / 255.0
            img = np.expand_dims(img, axis=0)
            pred_prob = model.predict(img)[0][0]
            pred_label = int(pred_prob > 0.5)
            results.append([row.image_id, pred_label])
            continue
    # If image not found or unreadable, default prediction to 0
    results.append([row.image_id, 0])

# Save results to CSV
submission = pd.DataFrame(results, columns=["image_id", "predicted_label"])
submission.to_csv(OUTPUT_CSV, index=False)

print(f"Submission saved to {OUTPUT_CSV}")
